In [127]:
import numpy as np
import cvxpy as cp
from basis_generator import rand_moment
import json

In [30]:
def single_behavior_visibility(X, X_basis):
    
    eta = cp.Variable((1, 1))
    alpha = cp.Variable((len(X_basis), 1))
    beta = cp.Variable((len(X_basis), 1))
    M = cp.Variable(X_basis[0].shape)
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1 - eta]

    for i in range(1,len(X)):
        constraints += [
            eta*X[i,i] + N[i,i] == M[i,i]
        ]

    constraints += [M >> 0]
    constraints += [M == sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [M[0,0] == 1]

    prob = cp.Problem(cp.Maximize(eta),
                      constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    coef = []
    for __ in range(3,len(constraints)-3):
        coef.append(np.real(constraints[__].dual_value[0][0]))
    
    return eta.value[0][0], coef

In [42]:
def NPA_bound(coef,X_basis):
    beta = cp.Variable((len(X_basis), 1))
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1]

    prob = cp.Problem(cp.Minimize(
            sum([N[j+1,j+1]*coef[j] for j in range(len(coef))])
            ),constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    return prob.value

In [27]:
# Import basis
basis_filename = "data_basis/2-dim-3-num_obs-2-len_seq-2-num_out.npy"
X_basis = np.load(basis_filename)

In [125]:
dimX = 3
dim_base = 2
num_obs = 3
len_seq = 2
num_out = 2
remove_last_out = True

In [32]:
X, rho, P = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
eta, coef = single_behavior_visibility(X, X_basis)

In [133]:
for __ in range(100):
    X_t, rho_t, P_t = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
    eta_t, coef_t = single_behavior_visibility(X_t, X_basis)
    if eta_t < eta:
        eta = eta_t
        coef = coef_t
        X = X_t
        rho = rho_t
        P = P_t

print(eta)

0.8417785173037825


In [134]:
Q = np.real(np.diag(X)[1:] @ coef)
print(Q)

-0.6372680915558379


In [135]:
C = NPA_bound(coef,X_basis)
print(C)

-0.47904664250464357


In [136]:
data = {}
# data["Moment Matrix"] = X
# data["Quantum State"] = rho
# data["Measurements"] = P
data["Inequality"] = coef
data["Classical Bound"] = C
data["Quantum Bound"] = Q

data["num of observables"] = num_obs
data["maximum length of sequences"] = len_seq
data["num of outcomes"] = num_out
data["dimension behavior"] = dimX
data["dimension base"] = dim_base

NAME = '{}-num_obs-{}-len_seq-{}-out_max-{}-dim_behavior-{}-dim_base'.format(num_obs, len_seq, num_out, dimX, dim_base)

with open("dimension_witness/" + NAME + '.json', 'w') as fp:
    json.dump(data, fp, indent=2)
    
np.save("dimension_witness/" + NAME + "-moment_matrix", X)
np.save("dimension_witness/" + NAME + "-state", rho)
np.save("dimension_witness/" + NAME + "-measurements", P)